In [86]:
from tensor import TensorSpec, Tensor

struct Image:
    var width: Int
    var height: Int
    var img_data: Tensor[DType.int16]
    var label: Int

    fn __init__(inout self, width: Int, height: Int, label: Int, data: Tensor[DType.int16]):
        self.width = width
        self.height = height
        let spec = TensorSpec(DType.int16, self.width, self.height)
        self.img_data = Tensor[DType.int16](spec)
        self.label = label

In [87]:
fn si8_to_ui8(int_tensor: Tensor[DType.int8]) raises -> Tensor[DType.uint8]:
    let spec = TensorSpec(DType.uint8, int_tensor.shape())
    var unsigned_ints = Tensor[DType.uint8](spec)
    for i in range(int_tensor.num_elements()):
        let signed_int = int_tensor[i]
        let unsigned_int = signed_int.cast[DType.uint8]()
        unsigned_ints[i] = unsigned_int
    return unsigned_ints

In [88]:
fn uint8_to_int32(uint8_tensor: Tensor[DType.uint8]) raises -> SIMD[DType.int32, 1]:
    let result = (
            (uint8_tensor[0].to_int() << 24) |
            (uint8_tensor[1].to_int() << 16) |
            (uint8_tensor[2].to_int() << 8) |
            uint8_tensor[3].to_int()
        )
    return SIMD[DType.int32, 1](result)

In [89]:
fn get_slice[type: DType](tensor: Tensor[type], start_index: Int, end_index: Int) raises -> Tensor[type]:
    if end_index < start_index:
        raise 'End index more than'
    elif end_index == start_index:
        var output_tensor = Tensor[type](1)
        output_tensor[0] = tensor[start_index]
        return output_tensor
    else:
        var output_tensor = Tensor[type](end_index - start_index)
        for i in range(start_index, end_index):
            output_tensor[i - start_index] = tensor[i]
        return output_tensor

In [90]:
fn read_data_as_images(images_path: Path) raises -> Tensor[DType.uint16]:
    if images_path.exists():

        let image_file = si8_to_ui8(images_path.read_bytes())
        let num_images = uint8_to_int32(get_slice[DType.uint8](image_file, 4, 8))
        let width = uint8_to_int32(get_slice[DType.uint8](image_file, 8, 12))
        let height = uint8_to_int32(get_slice[DType.uint8](image_file, 12, 16))

        let image_data = get_slice(image_file, 8, num_images.to_int() + 1)
        
        return 
    raise 'The images directory does not exist.'

error: Expression [89]:17:9: cannot implicitly convert 'None' value to 'Tensor[ui16]' in return value
        return 
        ^~~~~~

expression failed to parse (no further compiler diagnostics)

In [ ]:
fn read_data_as_labels(labels_path: Path) raises -> Tensor[DType.uint8]:
    if labels_path.exists():
        let labels = si8_to_ui8(labels_path.read_bytes())
        let num_labels = uint8_to_int32(get_slice[DType.uint8](labels, 4, 8))

        let label_data = get_slice(labels, 8, num_labels.to_int() + 1)
        
        return 
    raise 'The labels directory does not exist.'

In [91]:
let image_path = '/Users/tprazak/Documents/seminary_work_nn/MNIST_in_mojo/mnist/train-images.idx3-ubyte'
let labels_path = '/Users/tprazak/Documents/seminary_work_nn/MNIST_in_mojo/mnist/train-labels.idx1-ubyte'
let imgs_and_labels = read_data_as_images(image_path, labels_path)
let images = imgs_and_labels.get[0, Tensor[DType.uint16]]()


Tensor([[0, 0, 0, 28]], dtype=uint8, shape=4) Tensor([[0, 0, 0, 28]], dtype=uint8, shape=4)
28 28 60000


In [92]:
let hey = Tensor[DType.uint8](7)
hey[4] = 3
print(get_slice(hey, 2, 5))

Tensor([[0, 0, 3]], dtype=uint8, shape=3)
